In [1]:
from dotenv import load_dotenv
load_dotenv()

import kagglehub
import os
import numpy as np
import pandas as pd
import re
import shutil
from sqlalchemy import create_engine, types, text, Engine

from src import database_methods as dbm 


In [2]:
cache_path: str = kagglehub.dataset_download("gabrielramos87/an-online-shop-business")
os.listdir(cache_path)

['Sales Transaction v.4a.csv']

In [3]:
file_name: str = "Sales Transaction v.4a.csv"

target_dir: str = os.path.join("..", "data", "raw")
os.makedirs(target_dir, exist_ok=True)

source_file: str = os.path.join(cache_path, file_name)
destination_file: str = os.path.join(target_dir, file_name)

try:
    shutil.copy2(source_file, destination_file)
    print(f"Dataset successful copied into {target_dir} directory.")
except Exception as e:
    print(f"Error {e} - Unsuccessful dataset copying process.")

del target_dir, source_file, destination_file, cache_path


Dataset successful copied into ..\data\raw directory.


In [4]:

df: pd.DataFrame = pd.read_csv(
    "../data/raw/Sales Transaction v.4a.csv", 
    index_col=["TransactionNo"], 
    parse_dates=["Date"], 
    date_format="%m/%d/%Y", 
    dtype={
        "ProductNo": "category", 
        "ProductName": "category", 
        "Price": "float32", 
        "Quantity": "int32",  
        "CustomerNo": "Int32", 
        "Country": "category"
    }
)
df.index.name = "order_id"
df.rename(columns=lambda x: re.sub(r"([a-z])([A-Z])",r"\1_\2", x).lower().replace("_no", "_id"), inplace=True)
print(df.sample(5))

               date product_id                      product_name  price  \
order_id                                                                  
573345   2019-10-30      23489             Vintage Bells Garland  13.21   
536798   2018-12-02      21174  Pottering In The Shed Metal Sign  12.25   
C546167  2019-03-10      21527  Red Retrospot Traditional Teapot  18.40   
564760   2019-08-30      21775  Decorative Flore Bathroom Bottle  12.77   
580908   2019-12-06      21462     Nursery A B C Painted Letters   6.04   

          quantity  customer_id         country  
order_id                                         
573345           4        17609  United Kingdom  
536798          12        17838  United Kingdom  
C546167         -2        15611  United Kingdom  
564760           1        12760  United Kingdom  
580908           1        16495  United Kingdom  


In [5]:
print(f"NaN values quantity for each column:\n{df.isnull().sum()}")

NaN values quantity for each column:
date             0
product_id       0
product_name     0
price            0
quantity         0
customer_id     55
country          0
dtype: int64


In [6]:
df.dropna(how="any", inplace=True) # Due to records with missing value in "customer_no" are useless"
df["customer_id"] = df["customer_id"].astype("int32")

In [7]:
print(f"NaN values quantity for each column:\n{df.isnull().sum()}")

NaN values quantity for each column:
date            0
product_id      0
product_name    0
price           0
quantity        0
customer_id     0
country         0
dtype: int64


In [8]:
print(df.describe(include="all"))

                              date product_id  \
count                       536295     536295   
unique                         NaN       3767   
top                            NaN     85123A   
freq                           NaN       2378   
mean    2019-07-04 02:58:08.535228        NaN   
min            2018-12-01 00:00:00        NaN   
25%            2019-03-28 00:00:00        NaN   
50%            2019-07-20 00:00:00        NaN   
75%            2019-10-19 00:00:00        NaN   
max            2019-12-09 00:00:00        NaN   
std                            NaN        NaN   

                              product_name          price       quantity  \
count                               536295  536295.000000  536295.000000   
unique                                3767            NaN            NaN   
top     Cream Hanging Heart T-Light Holder            NaN            NaN   
freq                                  2378            NaN            NaN   
mean                           

In [9]:
mask: pd.Series = (df["quantity"] < 0) # Returns aren't taken into account
df = df[~mask]
print(f"Min value in 'quantity' column:\n{df["quantity"].min()}")

Min value in 'quantity' column:
1


In [10]:
print(f"Orders quantity for each customer:\n{df["customer_id"].value_counts()}")

Orders quantity for each customer:
customer_id
17841    7835
14911    5580
14096    5093
12748    4584
14606    2692
         ... 
18113       1
15070       1
14576       1
12791       1
17558       1
Name: count, Length: 4718, dtype: int64


In [11]:
print(df.columns.to_list())

['date', 'product_id', 'product_name', 'price', 'quantity', 'customer_id', 'country']


In [12]:
print(f"'product_id values max length:\n{df["product_id"].str.len().max()}")
print(f"'product_name' values max length:\n{df["product_name"].str.len().max()}")
print(f"'price' values max:\n{df["price"].max()}")
print(f"'quantity' values max:\n{df["quantity"].max()}")
print(f"'customer_id' values max:\n{df["customer_id"].max()}")
print(f"'country' values max length:\n{df["country"].str.len().max()}")


'product_id values max length:
6
'product_name' values max length:
35
'price' values max:
660.6199951171875
'quantity' values max:
80995
'customer_id' values max:
18287
'country' values max length:
20


In [13]:
try:
    engine: Engine = dbm.get_db_engine()
    print("DB Engine successfuly created")
except Exception as e:
    print(f"DB Engine creation error: {e}")
    
try:
    df.to_sql(
        name="e_commerce_order_details", 
        con=engine, 
        if_exists="replace", 
        index=True, 
        index_label="order_id", 
        method=dbm.psql_insert_copy, 
        chunksize=5000, 
        dtype={
            "order_id": types.VARCHAR(30), 
            "date": types.DATE, 
            "product_id": types.VARCHAR(30), 
            "product_name": types.VARCHAR(50), 
            "price": types.NUMERIC(8, 2), 
            "quantity": types.INT, 
            "customer_id": types.VARCHAR(30), 
            "country": types.VARCHAR(30), 
        }
    )
    print("DataFrame successfully migrated to postreSQL DB")
except Exception as e:
    print(f"DataFrame migration process error {e}")

try:
    with engine.connect() as connection:
        connection.execute(text("CREATE INDEX IF NOT EXISTS idx_customer_lookup ON e_commerce_order_details (customer_id);"))
        connection.commit()
    print("'order_id' successfully set as INDEX")
except Exception as e:
    print(f"INDEX set error: {e}")


DB Engine successfuly created
DataFrame successfully migrated to postreSQL DB
'order_id' successfully set as INDEX


In [14]:
sql_db_len: pd.DataFrame = pd.read_sql(
    sql="SELECT COUNT(order_id) AS rows_quantity FROM e_commerce_order_details;", 
    con=engine
)
print(f"DataFrame length:\n{len(df)}")
print(f"SQL DB length:\n{sql_db_len.iloc[0]}")
del sql_db_len

DataFrame length:
527764
SQL DB length:
rows_quantity    527764
Name: 0, dtype: int64


In [15]:
sql_db_random_row: pd.DataFrame = pd.read_sql(
    sql="SELECT * FROM e_commerce_order_details ORDER BY RANDOM() LIMIT 1", 
    con=engine
)
print(f"SQL DB random row:\n{sql_db_random_row}")
del sql_db_random_row

SQL DB random row:
  order_id        date product_id     product_name  price  quantity  \
0   554498  2019-05-24     47599A  Pink Party Bags   12.4         6   

  customer_id         country  
0       15314  United Kingdom  


In [16]:
df.to_parquet("../data/processed/e_commerce_order_details.parquet", engine="pyarrow")